## Mexican Broadsides Data Analysis, Batch 2

We need to do the same sorts of things we did the first time around. However, the workflow has since changed a bit. So we can't just basically re-run our notebook and scripts. So there will still be some additional exploration!  

### Once again, first thing's first: data conversion

We need to again transform the Excel data we got, that was then wrangled in OpenRefine to get reconciled to proper vocabularies. But this time, the data is an Excel sheet that is just bib numbers with FAST subjects. This is a bit different than last time, but will still involve a `csvkit` conversion:  

```
in2csv --sheet 'MexicanBroadsidesWithFastBatch0' MexicanBroadsidesWithFastBatch02.xlsx > mexican_broadsides_b2_FAST.csv
``` 

In [1]:
import pandas as pd
import os
import glob
import numpy as np

## Merging the data 

We have a (thankfully) simpler task than last time, which will save some time if it works. The idea is we take the wrangled subjects data, then merge it with the data we got from the DAMS' Roger export tool. It will merge on the 'bib' column which represents the bib number. First we'll import the Roger export data:

In [3]:
df1 = pd.read_csv('~/Downloads/broadsides2_no_subjs.csv')

In [4]:
df1.head()

,Object Unique ID,Level,File name,File use,Type of Resource,Language,Title,Subtitle,Translation,Variant,...,Person:1742-1815,Person:Creator,Person:author,Person:author.1,Person:author.2,Person:author.3,Person:author.4,Person:issuing body,Person:issuing body.1,Person:issuing body.2
0,N65537#16,Object,NaN,NaN,text,NaN,"Don Juan Ruiz de Apodaca y Eliza, Lopez de Let...",NaN,NaN,Por quanto al propio tiempo que causa una extr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Apodaca, Juan Ruiz de",NaN,NaN
1,N65537#152,Object,NaN,NaN,text,NaN,El Escmo. Sr. presidente de los Estados-Unidos...,NaN,NaN,"Secretaria de Hacienda, Departamento de Gobier...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Zavala, Lorenzo de, 1788-1836",NaN,NaN
2,N65537#17,Object,NaN,NaN,text,NaN,El Ecsmo. Sr. Presidente de los Estados Unidos...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Gómez Pedraza, Manuel, 1789-1851",NaN,NaN
3,N65537#151,Object,NaN,NaN,text,NaN,El Supremo Poder Ejecutivo me ha dirigido el D...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"García Illueca, José Ignacio, 1780-1832",NaN,NaN
4,N65537#18,Object,NaN,NaN,text,NaN,Don Francisco Xavier Venégas de Saavedra ... V...,NaN,NaN,Dedicado con la imparcialidad que me es caract...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since there's so many columns (49), we need to make sure the column we want to join ('bib') on exists in both dataFrames:

In [5]:
df1.columns

Index(['Object Unique ID', 'Level', 'File name', 'File use',
       'Type of Resource', 'Language', 'Title', 'Subtitle', 'Translation',
       'Variant', 'Variant.1', 'Variant.2', 'Corporate:Creator',
       'Corporate:Creator.1', 'Corporate:Creator.2',
       'Corporate:enacting juridiction', 'Corporate:enacting jurisdiciton',
       'Corporate:enacting jurisdiction', 'Corporate:issuing body',
       'Corporate:issuing body.1', 'Corporate:issuing body.2',
       'Corporate:issuing body.3', 'Corporate:isusing body', 'Date:creation',
       'Date:manufacture', 'Identifier:OCLC number', 'bib', 'Note:', 'Note:.1',
       'Note:.2', 'Note:.3', 'Note:.4', 'Note:.5', 'Note:.6', 'Note:.7',
       'Note:description', 'Note:material details',
       'Note:physical description', 'Note:publication', 'Person:1742-1815',
       'Person:Creator', 'Person:author', 'Person:author.1', 'Person:author.2',
       'Person:author.3', 'Person:author.4', 'Person:issuing body',
       'Person:issuing body.1', 

Now let's get our other dataFrame set up:

In [16]:
df2 = pd.read_csv('~/Downloads/mexican_broadsides_b2_FAST_cleaned.csv')

In [17]:
df2.head()

,bibex,bib,FAST Name,FAST Corporate,FAST Conference,FAST Topic,FAST Geographic,FAST Genre
0,b95834655,b9583465,NaN,NaN,NaN,Politics and government | Taxation--Law and le...,Mexico,NaN
1,b94184495,b9418449,NaN,NaN,NaN,Employees | Politics and government,Mexico,NaN
2,b95494273,b9549427,NaN,NaN,NaN,Politics and government,Mexico,NaN
3,b94319406,b9431940,NaN,NaN,NaN,Politics and government,Mexico,Sources
4,b96324739,b9632473,NaN,NaN,NaN,Politics and government | Smuggling--Law and l...,Mexico | New Spain,History | Sources


We can see the 'bib' column front and center here, as it is the only real identifier/key for the data

In [18]:
dff = pd.merge(left=df1,right=df2, how='left', left_on='bib', right_on='bib')

In [20]:
dff.head()

,Object Unique ID,Level,File name,File use,Type of Resource,Language,Title,Subtitle,Translation,Variant,...,Person:issuing body,Person:issuing body.1,Person:issuing body.2,bibex,FAST Name,FAST Corporate,FAST Conference,FAST Topic,FAST Geographic,FAST Genre
0,N65537#16,Object,NaN,NaN,text,NaN,"Don Juan Ruiz de Apodaca y Eliza, Lopez de Let...",NaN,NaN,Por quanto al propio tiempo que causa una extr...,...,"Apodaca, Juan Ruiz de",NaN,NaN,b96236826,NaN,NaN,Wars of Independence (Mexico : 1810-1821),Broadsides,NaN,NaN
1,N65537#152,Object,NaN,NaN,text,NaN,El Escmo. Sr. presidente de los Estados-Unidos...,NaN,NaN,"Secretaria de Hacienda, Departamento de Gobier...",...,"Zavala, Lorenzo de, 1788-1836",NaN,NaN,b9638489x,NaN,NaN,NaN,"Finance, Public--Law and legislation | Land te...",NaN,NaN
2,N65537#17,Object,NaN,NaN,text,NaN,El Ecsmo. Sr. Presidente de los Estados Unidos...,NaN,NaN,NaN,...,"Gómez Pedraza, Manuel, 1789-1851",NaN,NaN,b95779541,NaN,NaN,NaN,Mules,Mexico,NaN
3,N65537#151,Object,NaN,NaN,text,NaN,El Supremo Poder Ejecutivo me ha dirigido el D...,NaN,NaN,NaN,...,"García Illueca, José Ignacio, 1780-1832",NaN,NaN,b95005791,NaN,Mexico. Supremo Poder Ejecutivo (1823-1824),NaN,Monetary policy | Politics and government,Mexico,History | Sources
4,N65537#18,Object,NaN,NaN,text,NaN,Don Francisco Xavier Venégas de Saavedra ... V...,NaN,NaN,Dedicado con la imparcialidad que me es caract...,...,NaN,NaN,NaN,b96171078,NaN,NaN,NaN,Freedom of the press,NaN,History | Sources


This looks good, so let's export the merged data into a csv file:

In [21]:
# Uncomment the next line to actually generate the file
# dff.to_csv('~/Documents/mb2_wrangle.csv')

## Bringing the data back in and adding structure

I took the results of the above step and cleaned and normalized the data in OpenRefine. This was to fix typos, "normalize" values, which means just making them consistent, and merging or removing data as necessary. We can now import that as a dataFrame:

In [2]:
df3 = pd.read_csv('mexican_broadsides_batch2.csv')

In [4]:
df3.head()

,Object Unique ID,Level,File name,File use,Type of Resource,Language,Title,Variant,Variant.1,Variant.2,...,Person:author,Person:issuing body,Person:issuing body.1,bibex,FAST Name,FAST Corporate,FAST Conference,FAST Topic,FAST Geographic,FAST Genre
0,N65537#16,Object,NaN,NaN,text,spa - Spanish; Castilian,"Don Juan Ruiz de Apodaca y Eliza, Lopez de Let...",Por quanto al propio tiempo que causa una extr...,NaN,NaN,...,NaN,"Apodaca, Juan Ruiz de",NaN,b96236826,NaN,NaN,Wars of Independence (Mexico : 1810-1821),Broadsides,NaN,NaN
1,N65537#152,Object,NaN,NaN,text,spa - Spanish; Castilian,El Escmo. Sr. presidente de los Estados-Unidos...,"Secretaria de Hacienda, Departamento de Gobier...",NaN,NaN,...,NaN,"Zavala, Lorenzo de, 1788-1836",NaN,b9638489x,NaN,NaN,NaN,"Finance, Public--Law and legislation | Land te...",NaN,NaN
2,N65537#17,Object,NaN,NaN,text,spa - Spanish; Castilian,El Ecsmo. Sr. Presidente de los Estados Unidos...,NaN,NaN,NaN,...,NaN,"Gómez Pedraza, Manuel, 1789-1851",NaN,b95779541,NaN,NaN,NaN,Mules,Mexico,NaN
3,N65537#151,Object,NaN,NaN,text,spa - Spanish; Castilian,El Supremo Poder Ejecutivo me ha dirigido el D...,NaN,NaN,NaN,...,NaN,"García Illueca, José Ignacio, 1780-1832",NaN,b95005791,NaN,Mexico. Supremo Poder Ejecutivo (1823-1824),NaN,Monetary policy | Politics and government,Mexico,History | Sources
4,N65537#18,Object,NaN,NaN,text,spa - Spanish; Castilian,Don Francisco Xavier Venégas de Saavedra ... V...,Dedicado con la imparcialidad que me es caract...,NaN,NaN,...,NaN,NaN,NaN,b96171078,NaN,NaN,NaN,Freedom of the press,NaN,History | Sources


Now, for the other data, which has the "object structure", i.e., the objects each have rows for their files. That data, which basically just has the same IDs at object level, then filenames strcutured out, will need to be joined with the dataFrame we just made. Let's import the object structure:

In [5]:
df4 = pd.read_csv('~/Documents/mb_struct.csv')

In [7]:
df4[0:30]

,bibex,bib,leader,filename
0,b92888884,b9288888,0.0,b92888884_1.tif
1,b92888938,b9288893,0.0,b92888938_1.tif
2,b92889013,b9288901,NaN,NaN
3,NaN,NaN,0.0,b92889013_1.tif
4,NaN,NaN,1.0,b92889013_2.tif
5,b92889232,b9288923,0.0,b92889232_1.tif
6,b92901487,b9290148,0.0,b92901487_1.tif
7,b92905067,b9290506,0.0,b92905067_1.tif
8,b92905900,b9290590,0.0,b92905900_1.tif
9,b92982323,b9298232,0.0,b92982323_1.tif


In [8]:
dff = pd.merge(left=df4,right=df3, how='left', left_on='bib', right_on='bib')

In [9]:
dff

,bibex_x,bib,leader,filename,Object Unique ID,Level,File name,File use,Type of Resource,Language,...,Person:author,Person:issuing body,Person:issuing body.1,bibex_y,FAST Name,FAST Corporate,FAST Conference,FAST Topic,FAST Geographic,FAST Genre
0,b92888884,b9288888,0.0,b92888884_1.tif,N65537#60,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Ortiz de la Torre, Manuel",NaN,b92888884,"Bustamante, Anastasio, 1780-1853",NaN,NaN,Politics and government,Mexico,NaN
1,b92888938,b9288893,0.0,b92888938_1.tif,N65537#137,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Alamán, Lucas, 1792-1853",NaN,b92888938,NaN,NaN,NaN,Amnesty | Insurgency | Politics and government,"Mexico | Campeche (Campeche, Mexico) | Guadala...",Sources
2,b92889013,b9288901,NaN,NaN,N65537#52,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Alamán, Lucas, 1792-1853",NaN,b92889013,NaN,NaN,NaN,Diplomatic relations | Politics and government,Mexico | Spain,History | Sources
3,NaN,NaN,0.0,b92889013_1.tif,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1.0,b92889013_2.tif,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,b92889232,b9288923,0.0,b92889232_1.tif,N65537#206,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Alamán, Lucas, 1792-1853",NaN,b92889232,NaN,NaN,NaN,NaN,Mexico,History | Sources
6,b92901487,b9290148,0.0,b92901487_1.tif,N65537#61,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Alamán, Lucas, 1792-1853",NaN,b92901487,NaN,NaN,NaN,Medical education,Mexico,History | Sources
7,b92905067,b9290506,0.0,b92905067_1.tif,N65537#146,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Bustamante, Anastasio, 1780-1853","Alamán, Lucas, 1792-1853",b92905067,NaN,NaN,NaN,Epidemics | Public health,Mexico City (Mexico),NaN
8,b92905900,b9290590,0.0,b92905900_1.tif,N65537#149,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Bustamante, Anastasio, 1780-1853","Alamán, Lucas, 1792-1853",b92905900,NaN,NaN,NaN,Elections | Politics and government,Mexico | Jalisco (Mexico),NaN
9,b92982323,b9298232,0.0,b92982323_1.tif,N65537#25,Object,NaN,NaN,text,spa - Spanish; Castilian,...,NaN,"Victoria, Guadalupe, 1786-1843","Espinosa de los Monteros, Juan José",b92982323,NaN,NaN,NaN,Administrative and political divisions,Mexico | Mexico City (Mexico),History | Sources


In [10]:
# Uncomment to actually make file
# dff.to_csv('~/Documents/mb2_olr.csv')